In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 61.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=bd802d8cc1d2a67612fdf6f07a8d7a8951cb5ae28bfd0c9f0f650688bb134fee
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
!wget http://www.jeuxdemots.org/JDM-LEXICALNET-FR/09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt.zip
!unzip 09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt.zip  -d ./
!iconv -f CP1252 -t UTF-8 /content/JDM-LEXICALNET-FR/09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt -o /content/JDM-LEXICALNET-FR/09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML-UTF8.txt

--2021-10-21 13:02:27--  http://www.jeuxdemots.org/JDM-LEXICALNET-FR/09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt.zip
Resolving www.jeuxdemots.org (www.jeuxdemots.org)... 193.49.110.106
Connecting to www.jeuxdemots.org (www.jeuxdemots.org)|193.49.110.106|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1930689580 (1.8G) [application/zip]
Saving to: ‘09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt.zip’

09032019-LEXICALNET 100%[===================>]   1.80G  29.4MB/s    in 65s     

2021-10-21 13:03:31 (28.5 MB/s) - ‘09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt.zip’ saved [1930689580/1930689580]

Archive:  09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt.zip
  inflating: ./JDM-LEXICALNET-FR/09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML.txt  


In [3]:
import sys
import os
import pyspark as ps
import warnings
import re
import datetime

from pyspark import SparkContext 
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StringType

In [71]:
spark = SparkSession.Builder().appName("RezoJDM-SDS").getOrCreate()
sc = spark.sparkContext

jdmDUMP = sc.textFile("/content/JDM-LEXICALNET-FR/09032019-LEXICALNET-JEUXDEMOTS-FR-NOHTML-UTF8.txt")
# jdmDUMP = sc.textFile("/content/09032019-Template.txt")

In [72]:
jdmNodes = jdmDUMP.filter(lambda x: x[:4] == "eid=")

jdmNodes = jdmNodes.map(lambda x: 
                        Row(
                                eid = int(re.findall("eid=(\d+)\|n=", x)[0]) ,
                                n = re.findall("n=\"(.+)\"\|t=", x)[0] ,
                                t = int(re.findall("t=(-?\d+)\|w=", x)[0]) ,
                                w = int(re.findall("w=(-?\d+)", x)[0])
                            )
                  )

# only type 1 and 2 should be kept
jdmNodes = jdmNodes.filter(lambda x: (x["t"] == 1) | (x["t"] == 2))

# node whose text starts and ends with equal (“=”) are not french, don’t keep them
jdmNodes = jdmNodes.filter(lambda x: (~((x["n"][0] == "=") & (x["n"][-1] == "="))))

# filter the weights lesser than M(>50)
jdmNodes = jdmNodes.filter(lambda x: x["w"] > 50)

dfNodes = jdmNodes.toDF()

jdmNodes.count()

73106

In [73]:
jdmRelations = jdmDUMP.filter(lambda x: x[:4] == "rid=")

jdmRelations = jdmRelations.map(lambda x: 
                                    Row(
                                            rid = int(re.findall("rid=(\d+)\|n1=", x)[0]) ,
                                            n1 = int(re.findall("n1=(\d+)\|n2=", x)[0]) ,
                                            n2 = int(re.findall("n2=(-?\d+)\|t=", x)[0]) ,
                                            t = int(re.findall("t=(-?\d+)\|w=", x)[0]) ,
                                            w = int(re.findall("w=(-?\d+)", x)[0])
                                        )
                            )
# only type 1 and 2 should be kept
jdmRelations = jdmRelations.filter(lambda x: x["w"] > 50)

# keep all nodes except { 12, 18, 29, 33, 36, 45, 46, 47, 48, 1000, 1001, 1002, 444, 128, 200}
black_list_relation_ids = [12, 18, 29, 33, 36, 45, 46, 47, 48, 1000, 1001, 1002, 444, 128, 200]
jdmRelations = jdmRelations.filter(lambda x: x["t"] not in black_list_relation_ids)

# edges might refer to non-existing node, don’t keep those edges
existing_node_ids = list(dfNodes.toPandas()["eid"])
jdmRelations = jdmRelations.filter(lambda x: (x["n1"] in existing_node_ids) & (x["n2"] in existing_node_ids))

jdmRelations.count()

1512986

In [74]:
dfNodes = jdmNodes.toDF()
dfNodes = dfNodes.toPandas()

dfRelations = jdmRelations.toDF()
dfRelations = dfRelations.toPandas()

In [75]:
dfNodes.to_csv("nodes.csv", index=False)
dfRelations.to_csv("relations.csv", index=False)

In [120]:
dfNodes

,eid,n,t,w
0,1,vraiment,1,144
1,4,Conscience,1,52
2,6,provenir,1,52
3,8,traversée,1,158
4,12,discréditer,1,76
...,...,...,...,...
73101,11151763,gros coup,1,52
73102,11172658,oeuf>140176,1,72
73103,11303561,opéra ballet,1,56
73104,11438901,être vautré,1,52


In [79]:
dfRelations

,rid,n1,n2,t,w
0,1,70226,132763,0,60
1,2,70226,55756,0,60
2,3,70226,73274,0,60
3,4,82745,95245,0,150
4,5,109319,33319,0,60
...,...,...,...,...,...
1512981,338665982,68439,21301,24,63
1512982,338665983,21301,68439,13,60
1512983,338665988,396247,21301,24,58
1512984,338665989,21301,396247,13,55


In [76]:
from google.colab import files

files.download("nodes.csv")
files.download("relations.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
nodes_out_degree = {}
nodes_in_degree = {}
nodes_degree = {}

for _, row in dfRelations.iterrows():
    
    if row.n1 in nodes_out_degree:
        nodes_out_degree[row.n1]+=1
    else:
        nodes_out_degree[row.n1]=1


    if row.n2 in nodes_in_degree:
        nodes_in_degree[row.n2]+=1
    else:
        nodes_in_degree[row.n2]=1


    if row.n1 in nodes_degree:
        nodes_degree[row.n1]+=1
    else:
        nodes_degree[row.n1]=1


    if row.n2 in nodes_degree:
        nodes_degree[row.n2]+=1
    else:
        nodes_degree[row.n2]=1


In [121]:
nodes_degree

{70226: 24,
 132763: 107,
 55756: 398,
 73274: 9,
 82745: 82,
 95245: 1187,
 109319: 32,
 33319: 54,
 32912: 77,
 102633: 681,
 97688: 22,
 136060: 15,
 131749: 32,
 123796: 16,
 30539: 13,
 45844: 225,
 64840: 11,
 130400: 2059,
 35444: 14,
 33187: 43,
 1453: 61,
 35841: 802,
 79684: 586,
 122384: 57,
 96889: 7,
 97756: 1013,
 49175: 147,
 63235: 1933,
 68299: 3,
 56280: 390,
 67400: 35,
 17559: 6343,
 12887: 122,
 71348: 414,
 136472: 246,
 11849: 120,
 129498: 97,
 7766: 103,
 2808: 66,
 36222: 193,
 29592: 3605,
 136729: 5,
 58324: 8,
 30030: 765,
 136733: 11,
 23519: 39,
 79239: 109,
 122704: 251,
 115865: 529,
 61092: 73,
 71578: 791,
 36899: 459,
 90075: 843,
 60829: 349,
 52747: 3,
 112601: 27,
 84773: 34,
 28947: 7,
 68307: 43,
 3695: 687,
 55593: 7,
 26846: 181,
 61147: 475,
 43583: 48,
 49345: 24,
 74888: 17,
 95280: 135,
 114738: 392,
 6334: 127,
 61686: 125,
 127337: 70,
 23705: 19,
 118876: 251,
 5978: 7,
 107477: 655,
 18229: 3,
 68255: 65,
 131321: 60,
 136736: 70,
 697

In [109]:
len(dfRelations["t"].unique())

113

In [116]:
values=[]
keys=[]

for key, value in nodes_degree.items():
    values.append(value)
    keys.append(key)
    if value == 15818:
        print(key)

116477


In [117]:
import numpy as np
min(items), max(items), np.mean(items)

(1, 15818, 42.73066440725835)

In [119]:
dfNodes[dfNodes["eid"]==116477]

,eid,n,t,w
35977,116477,personne,1,9834
